In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install music21

In [ ]:
!nvidia-smi

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import glob,pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils

# Preprocessing

In [ ]:
%cd /content/drive/MyDrive/Project/Compose/1_LSTM/ #절대경로지정

In [ ]:
# MIDI 파일을 잘 불러왔는지 테스트
# MIDI 파일을 불러오는 함수
midi = converter.parse("./data/lstm_songs/btv1.mid") # Midi file 경로 입니다.
# MIDI 파일 내의 notes(음정, 박자를 포함하는 정보)를 불러온다
notes_to_parse = midi.flat.notes
# 불러온 notes의 갯수
print(np.shape(notes_to_parse))
# 10개 테스트 출력
for e in notes_to_parse[:20]:
  print(e, e.offset)
# Note / Chord 두 종류로 나뉜다, Chord는 Note의 집합이다

In [ ]:
import music21

In [ ]:
notes = []

for i,file in enumerate(glob.glob("./data/lstm_songs/*.mid")): # Midi file 경로 입니다.
  midi = converter.parse(file) 
  print('\r', 'Parsing file ', i, " ",file, end='') 

  notes_to_parse = None


  # MIDI 파일의 Note / Chord / Tempo 정보만 가져온다
  try: 
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
    
  except: 
    notes_to_parse = midi.flat.notes

  # Note / Chord / Tempo 정보 중 Note, Chord 의 경우 따로 처리
  for e in notes_to_parse:
    # Note 인 경우 높이(Pitch), 옥타브로 저장
    if isinstance(e, music21.note.Note):
      notes.append(str(e.pitch))
    # Chord 인 경우 각 Note의 음높이(Pitch)를 '.'으로 나누어 저장
    elif isinstance(e, music21.chord.Chord):
      notes.append('.'.join(str(n) for n in e.normalOrder))

In [ ]:
n_vocab = (len(set(notes)))
print('Classes of notes : ', n_vocab, '\n')
print('notes : ', notes[:500])
print('length of notes : ', len(notes), '\n')


pitchnames = sorted(set(item for item in notes))
print('pitchnames : ', pitchnames)
print('length of pitchnames : ', len(pitchnames), '\n')

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print('note_to_int : ', note_to_int)

In [ ]:
# LSTM 모델을 위한 Training Dataset 생성
seq_len = 100 

net_in = []
net_out = []


for i in range(0, len(notes) - seq_len):
  seq_in = notes[i:i + seq_len] 
  seq_out = notes[i + seq_len]  
  net_in.append([note_to_int[char] for char in seq_in]) 
  net_out.append(note_to_int[seq_out]) 

print(np.shape(net_in))
print(np.shape(net_out))

In [ ]:
# LSTM 모델 입출력에 맞게 Dataset 전처리

n_patterns = len(net_in)
print('n_patterns : ', n_patterns)


# LSTM 입력에 맞는 모양으로 바꿔준다 
net_in = np.reshape(net_in, (n_patterns, seq_len, 1))
print('shape of net_in : ', net_in.shape)


net_in = net_in / float(n_vocab)
net_out = np_utils.to_categorical(net_out)
print('shape of net_out : ', net_out.shape)

# Modeling

In [ ]:
data_dim = net_in.shape[2]

model = Sequential()
model.add(LSTM(512, input_shape=(seq_len, data_dim), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(256)) # Layer추가 (2개로 학습 안됨)
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [ ]:
model.fit(net_in, net_out, epochs=10, batch_size=32)

# Predict

In [ ]:
# 작곡을 위해 LSTM 모델 입력

net_in = []
output = []
for i in range(0, len(notes) - seq_len, 1):
  seq_in = notes[i:i + seq_len]
  seq_out = notes[i + seq_len]
  net_in.append([note_to_int[char] for char in seq_in])
  output.append(note_to_int[seq_out])
  
n_patterns = len(net_in)

In [ ]:
# pattern : Dataset의 입력 전체 시퀀스 중 랜덤하게 고른 시퀀스
start = np.random.randint(0, len(net_in)-1)
pattern = net_in[start]
print('Random Sequence : ', pattern)

# int_to_note: 정수를 다시 Note로 바꾸기 위한 dictionary 자료형
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print('int_to_note : ', int_to_note)

In [ ]:
pred_out = []
# generate 500 notes
for i in range(0, 500):
  pred_in = np.reshape(pattern, (1, len(pattern), 1))
  pred_in = pred_in / float(n_vocab)

  prediction = model.predict(pred_in, verbose=0)
  index = np.argmax(prediction)
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  
  pred_out.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

In [ ]:
print('length of pred_out : ', len(pred_out))
print('pred_out : ', pred_out)

# Output

In [ ]:
# LSTM 모델이 예측한 값들로부터 MIDI 파일을 만들어준다
offset = 0 

output_notes = []
for pattern in pred_out:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.') 
        notes = [] 
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) 
            new_note.storedInstrument = instrument.Piano() # 악기 정보 설정,피아노 음만 추출
            notes.append(new_note) 

        new_chord = chord.Chord(notes)
        new_chord.offset = offset 
        output_notes.append(new_chord)
    else:
        new_note = music21.note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5 # 박자고정

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='./Output/test_output.mid')

> Beethoven LSTM COMPOSE <br>
> Basecode FROM <br>
> https://bcdeep.tistory.com/10 <br>
> https://tykimos.github.io/2018/09/14/How_to_Generate_Music_using_a_LSTM_Neural_Network_in_Keras/